In [2]:
import numpy as np
import pandas as pd
from aif360.datasets import CompasDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import BinaryLabelDatasetMetric

In [1]:
%matplotlib inline
# Load all necessary packages
# import sys
# sys.path.append("../")
import numpy as np
from tqdm import tqdm

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

from common_utils import compute_metrics

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2024-01-02 21:08:32.216888: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 21:08:32.216920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 21:08:32.217690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 21:08:32.221706: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical o

In [3]:
import itertools

import numpy as np
import pandas as pd

from aif360.metrics import BinaryLabelDatasetMetric, MDSSClassificationMetric
from aif360.detectors import bias_scan

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas

In [12]:
# Load the dataset with 'sex' and 'race' as protected attributes
compas_data = CompasDataset(
    protected_attribute_names=['sex', 'race'],
    privileged_classes=[['Male'], ['Caucasian']],  # 'Male' for 'sex' and 'Caucasian' for 'race'
    features_to_drop=[]  # No features to drop initially
)

# Filter the dataset for individuals aged less than 25
compas_data = compas_data.subset(compas_data.features[:, compas_data.feature_names.index('age')] < 25)

# Define privileged and unprivileged groups for both 'sex' and 'race'
privileged_groups = [{'sex': 1, 'race': 1}]
unprivileged_groups = [{'sex': 0, 'race': 0}]

In [13]:
# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_vt = compas_data.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = compas_data.split([0.3], shuffle=True)

In [14]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(942, 401)


#### Favorable and unfavorable labels

0.0 1.0


#### Protected attribute names

['sex', 'race']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'age_cat=25 - 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'c_charge_degree=F', 'c_charge_degree=M', 'c_charge_desc=Abuse Without Great Harm', 'c_charge_desc=Agg Abuse Elderlly/Disabled Adult', 'c_charge_desc=Agg Assault W/int Com Fel Dome', 'c_charge_desc=Agg Battery Grt/Bod/Harm', 'c_charge_desc=Agg Fleeing and Eluding', 'c_charge_desc=Agg Fleeing/Eluding High Speed', 'c_charge_desc=Aggr Child Abuse-Torture,Punish', 'c_charge_desc=Aggrav Battery w/Deadly Weapon', 'c_charge_desc=Aggrav Child Abuse-Agg Battery', 'c_charge_desc=Aggrav Child Abuse-Causes Harm', 'c_charge_desc=Aggrav Stalking After Injunctn', 'c_charge_desc=Aggravated Assault', 'c_charge_desc=Aggravated Assault W/Dead Weap', 'c_charge_desc=Aggravated Assault W/dead Weap', 'c_charge_desc=Aggravated Assault W/o Firearm', 'c_charge_desc=Aggravated Assault w/Firearm', 'c_charge_desc=Aggravated Battery', 'c_charge_desc=Aggra

In [15]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.075624


In [16]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(dataset_orig_train)
dataset_transf_train = RW.transform(dataset_orig_train)

In [17]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


In [18]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()
w_train = dataset_orig_train.instance_weights.ravel()

lmod = LogisticRegression()
lmod.fit(X_train, y_train, 
         sample_weight=dataset_orig_train.instance_weights)
y_train_pred = lmod.predict(X_train)

# positive class index
pos_ind = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]

dataset_orig_train_pred = dataset_orig_train.copy()
dataset_orig_train_pred.labels = y_train_pred

In [19]:
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
X_valid = scale_orig.transform(dataset_orig_valid_pred.features)
y_valid = dataset_orig_valid_pred.labels
dataset_orig_valid_pred.scores = lmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)
print(dataset_orig_valid_pred.scores[:10])

dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = scale_orig.transform(dataset_orig_test_pred.features)
y_test = dataset_orig_test_pred.labels
dataset_orig_test_pred.scores = lmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

[[0.11128556]
 [0.89134124]
 [0.73671239]
 [0.22525355]
 [0.33407149]
 [0.7123185 ]
 [0.39859159]
 [0.09899446]
 [0.79964556]
 [0.00458024]]


In [20]:
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = dataset_orig_valid_pred.scores > class_thresh
    dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
    dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(dataset_orig_valid,
                                             dataset_orig_valid_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    
    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()\
                       +classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no reweighing) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no reweighing) = %.4f" % best_class_thresh)

Best balanced accuracy (no reweighing) = 0.7175
Optimal classification threshold (no reweighing) = 0.4456


In [21]:
display(Markdown("#### Predictions from original testing data"))
bal_acc_arr_orig = []
disp_imp_arr_orig = []
avg_odds_diff_arr_orig = []

print("Classification threshold used = %.4f" % best_class_thresh)
for thresh in tqdm(class_thresh_arr):
    
    if thresh == best_class_thresh:
        disp = True
    else:
        disp = False
    
    fav_inds = dataset_orig_test_pred.scores > thresh
    dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
    dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label
    
    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                                      unprivileged_groups, privileged_groups,
                                      disp = disp)

    bal_acc_arr_orig.append(metric_test_bef["Balanced accuracy"])
    avg_odds_diff_arr_orig.append(metric_test_bef["Average odds difference"])
    disp_imp_arr_orig.append(metric_test_bef["Disparate impact"])

#### Predictions from original testing data

Classification threshold used = 0.4456


100%|██████████| 100/100 [00:00<00:00, 928.49it/s]

Balanced accuracy = 0.6982
Statistical parity difference = 0.2117
Disparate impact = 1.3685
Average odds difference = 0.1693
Equal opportunity difference = 0.1702
Theil index = 0.2009


## Bias Scan

In [39]:
# Convert the test dataset to DataFrame and add actual observed labels
df_test, _ = dataset_orig_test.convert_to_dataframe()
df_test['observed'] = df_test['two_year_recid']

# Use the scores from dataset_orig_test_pred as probabilities for the test dataset
df_test['probabilities'] = dataset_orig_test_pred.scores

# Perform bias scan on the test dataset
privileged_subset = bias_scan(df_test[['sex', 'age', 'race']], df_test['observed'], df_test['probabilities'],
                              favorable_value=dataset_orig_test_pred.favorable_label,
                              penalty=0.5, overpredicted=True)
unprivileged_subset = bias_scan(df_test[['sex', 'age', 'race']], df_test['observed'], df_test['probabilities'],
                                favorable_value=dataset_orig_test_pred.favorable_label,
                                penalty=0.5, overpredicted=False)

# Display results
print("Privileged Subset:", privileged_subset)
print("Unprivileged Subset:", unprivileged_subset)

Privileged Subset: ({'age': [19.0]}, 2.1428)
Unprivileged Subset: ({'age': [21.0, 23.0], 'race': [0.0]}, 2.1571)


## Measuring and Comparing Bias

Compare subset age == 19

In [51]:
# Load COMPAS dataset without specifying protected attributes related to 'sex' or 'race'
compas_data = CompasDataset()

# Subset dataset for individuals aged 19
age_19_subgroup = compas_data.subset(compas_data.features[:, compas_data.feature_names.index('age')] == 19)

# Dummy privileged and unprivileged groups (needed for metric calculation)
dummy_privileged_groups = [{'sex': 1}]
dummy_unprivileged_groups = [{'sex': 0}]

# Metric for the age 19 subgroup
metric_age_19 = BinaryLabelDatasetMetric(age_19_subgroup, 
                                         unprivileged_groups=dummy_unprivileged_groups,
                                         privileged_groups=dummy_privileged_groups)

# Display bias metrics
print("Mean Difference (Age 19): ", metric_age_19.mean_difference())

Mean Difference (Age 19):  0.05


In [53]:
# Create boolean arrays for age and race conditions
age_condition = (compas_data.features[:, compas_data.feature_names.index('age')] == 21) | \
                (compas_data.features[:, compas_data.feature_names.index('age')] == 23)
race_condition = compas_data.features[:, compas_data.feature_names.index('race')] == 0

# Combine conditions and subset the dataset
combined_condition = age_condition & race_condition
age_21_23_subgroup = compas_data.subset(combined_condition)

# Dummy privileged and unprivileged groups for metric calculation
dummy_privileged_groups = [{'sex': 1}]
dummy_unprivileged_groups = [{'sex': 0}]

# Metric for the age 21 and 23, non-Caucasian subgroup
metric_age_21_23 = BinaryLabelDatasetMetric(age_21_23_subgroup, 
                                            unprivileged_groups=dummy_unprivileged_groups,
                                            privileged_groups=dummy_privileged_groups)

# Display bias metrics
print("Mean Difference (Age 21 and 23, non-Caucasian): ", metric_age_21_23.mean_difference())

Mean Difference (Age 21 and 23, non-Caucasian):  -0.14097476978363405
